In [ ]:
import os
import avstack
import avapi
import numpy as np
import quaternion

%load_ext autoreload
%autoreload 2

obj_data_dir_k = "../../data/KITTI/object"
raw_data_dir_k = "../../data/KITTI/raw"

KOD = avapi.kitti.KittiObjectDataset
KRD = avapi.kitti.KittiRawDataset(raw_data_dir_k)
KSM = avapi.kitti.KittiScenesManager(obj_data_dir_k)

## Test Visualization

In [ ]:
# Single frame visualization
KDM = KSM.get_scene_dataset_by_index(0)
frame = KDM.frames[30]
img = KDM.get_image(frame, sensor="image-2")
pc = KDM.get_lidar(frame)
objects = KDM.get_objects(frame)
avapi.visualize.snapshot.show_objects_on_image(
    img, objects, projection="3d", inline=True
)
avapi.visualize.snapshot.show_lidar_on_image(pc, img)

## Test Detection

In [ ]:
model = "2d-img"
if model == "2d-img":
    M = avstack.modules.perception.object2dfv.MMDetObjectDetector2D(
        model="fasterrcnn", dataset="kitti", gpu=1
    )
elif model == "3d-img":
    M = avstack.modules.perception.object3d.MMDetObjectDetector3D(
        model="pgd", dataset="kitti", gpu=0
    )
elif model == "3d-lidar":
    M = avstack.modules.perception.object3d.MMDetObjectDetector3D(
        model="pointpillars", dataset="kitti", gpu=1
    )
else:
    raise NotImplementedError(model)

In [ ]:
frame = KDM.frames[30]
img = KDM.get_image(frame, sensor="image-2")
pc = KDM.get_lidar(frame)
objects = KDM.get_objects(frame)

if model == "2d-img":
    outputs = M(img, identifier="test", frame=frame)
elif model == "3d-img":
    outputs = M(img, identifier="test", frame=frame)
elif model == "3d-lidar":
    outputs = M(pc, identifier="test", frame=frame)
else:
    raise NotImplementedError(model)
avapi.visualize.snapshot.show_image_with_boxes(img, outputs.data, inline=True)